In [27]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm



In [2]:
from datetime import date, timedelta
yesterday = date.today() - timedelta(days = 1)

In [5]:
q = '''
SELECT i.ref_term, count(*)
FROM bun_log_db.app_event_type_impression i
JOIN
  (SELECT ref_term
   FROM bun_log_db.app_event_type_impression
   WHERE ref_term IS NOT NULL
     AND device_type IN ('a')
     AND YEAR||MONTH||DAY = {}
   GROUP BY 1
   ORDER BY count(*) DESC
   LIMIT 1000) t ON i.ref_term = t.ref_term
WHERE i.year||i.month||i.day = {}
  AND i.device_type IN ('a')
  AND page_id = '검색결과'
GROUP BY 1
'''.format('\'' + yesterday.strftime('%Y%m%d') + '\'', '\'' + yesterday.strftime('%Y%m%d') + '\'')
top_k_keywords = pd.read_sql(q, con=bun_dw)

In [6]:
top_k_keywords

,ref_term,count
0,스투시,33467
1,바이크,11253
2,kf94마스크,19763
3,롤,26842
4,285,14630
5,캐릭캐릭체인지,24123
6,잇미샤,9803
7,디지몬,25742
8,지고트,8504
9,자켓,23670


In [42]:
top_k_keywords[top_k_keywords['ref_term'] == '자켓'].index.tolist()

[9]

In [88]:
pu_keywords = {}
for i in tqdm(range(len(top_k_keywords['ref_term'].tolist())), desc='collecting'):
    keyword = top_k_keywords.loc[i, 'ref_term']
    q = '''
    SELECT split_part(ad_ref, '_', 4) as pu_id,
           split_part(ad_ref, '_', 5) as bid,
           count(*) as cnt
    FROM bun_log_db.ca_event_type_item_click_v5
    WHERE YEAR||MONTH||DAY = {:}
      AND ad_ref LIKE {:}
      AND keyword LIKE {:}
    GROUP BY 1, 2
    '''.format('\'' + yesterday.strftime('%Y%m%d') + '\'', '\'ad_power_up%%\'', '\'%%' + keyword.strip() + '%%\'' )
    temp_df = pd.read_sql(q, con=bun_dw)
    if len(temp_df) > 0:
        pu_keywords[keyword.strip()] = temp_df
        pu_keywords[keyword.strip()]['imp_cnt'] = top_k_keywords[top_k_keywords['ref_term'] == keyword]['count'].values[0]
        pu_keywords[keyword.strip()]['rank'] = top_k_keywords[top_k_keywords['ref_term'] == keyword].index[0]










collecting:   0%|          | 0/1000 [00:00<?, ?it/s]








collecting:   0%|          | 1/1000 [00:07<2:05:53,  7.56s/it]








collecting:   0%|          | 2/1000 [00:15<2:06:42,  7.62s/it]








collecting:   0%|          | 3/1000 [00:22<2:05:53,  7.58s/it]








collecting:   0%|          | 4/1000 [00:30<2:04:19,  7.49s/it]








collecting:   0%|          | 5/1000 [00:38<2:07:37,  7.70s/it]








collecting:   1%|          | 6/1000 [00:46<2:08:12,  7.74s/it]








collecting:   1%|          | 7/1000 [00:54<2:12:51,  8.03s/it]








collecting:   1%|          | 8/1000 [01:07<2:33:59,  9.31s/it]








collecting:   1%|          | 9/1000 [01:15<2:27:04,  8.90s/it]








collecting:   1%|          | 10/1000 [01:22<2:20:12,  8.50s/it]








collecting:   1%|          | 11/1000 [01:31<2:23:01,  8.68s/it]








collecting:   1%|          | 12/1000 [01:39<2:19:34,  8.48s/it]








collecting:   1%|▏         | 13/1000 [01:46<2:10:54,  7.96s/it]








col

collecting:   8%|▊         | 82/1000 [11:05<2:03:50,  8.09s/it]








collecting:   8%|▊         | 83/1000 [11:13<2:03:32,  8.08s/it]








collecting:   8%|▊         | 84/1000 [11:22<2:08:19,  8.41s/it]








collecting:   8%|▊         | 85/1000 [11:32<2:14:40,  8.83s/it]








collecting:   9%|▊         | 86/1000 [11:41<2:16:59,  8.99s/it]








collecting:   9%|▊         | 87/1000 [11:50<2:15:44,  8.92s/it]








collecting:   9%|▉         | 88/1000 [12:00<2:20:07,  9.22s/it]








collecting:   9%|▉         | 89/1000 [12:09<2:18:21,  9.11s/it]








collecting:   9%|▉         | 90/1000 [12:18<2:17:42,  9.08s/it]








collecting:   9%|▉         | 91/1000 [12:27<2:15:38,  8.95s/it]








collecting:   9%|▉         | 92/1000 [12:36<2:16:08,  9.00s/it]








collecting:   9%|▉         | 93/1000 [12:44<2:13:43,  8.85s/it]








collecting:   9%|▉         | 94/1000 [12:53<2:11:51,  8.73s/it]








collecting:  10%|▉         | 95/1000 [13:01<2:10:39,  8.66s/it]


collecting:  16%|█▋        | 164/1000 [21:59<1:48:46,  7.81s/it]








collecting:  16%|█▋        | 165/1000 [22:07<1:49:49,  7.89s/it]








collecting:  17%|█▋        | 166/1000 [22:15<1:49:25,  7.87s/it]








collecting:  17%|█▋        | 167/1000 [22:23<1:50:52,  7.99s/it]








collecting:  17%|█▋        | 168/1000 [22:31<1:50:21,  7.96s/it]








collecting:  17%|█▋        | 169/1000 [22:38<1:48:10,  7.81s/it]








collecting:  17%|█▋        | 170/1000 [22:48<1:53:31,  8.21s/it]








collecting:  17%|█▋        | 171/1000 [22:56<1:55:41,  8.37s/it]








collecting:  17%|█▋        | 172/1000 [23:05<1:56:27,  8.44s/it]








collecting:  17%|█▋        | 173/1000 [23:16<2:05:20,  9.09s/it]








collecting:  17%|█▋        | 174/1000 [23:25<2:05:08,  9.09s/it]








collecting:  18%|█▊        | 175/1000 [23:35<2:10:37,  9.50s/it]








collecting:  18%|█▊        | 176/1000 [23:45<2:13:39,  9.73s/it]








collecting:  18%|█▊        | 177/1000 [23:57<2:20:0

collecting:  25%|██▍       | 246/1000 [35:17<1:47:25,  8.55s/it]








collecting:  25%|██▍       | 247/1000 [35:25<1:43:13,  8.23s/it]








collecting:  25%|██▍       | 248/1000 [35:32<1:41:21,  8.09s/it]








collecting:  25%|██▍       | 249/1000 [35:41<1:42:23,  8.18s/it]








collecting:  25%|██▌       | 250/1000 [35:49<1:43:53,  8.31s/it]








collecting:  25%|██▌       | 251/1000 [35:57<1:41:37,  8.14s/it]








collecting:  25%|██▌       | 252/1000 [36:04<1:38:24,  7.89s/it]








collecting:  25%|██▌       | 253/1000 [36:12<1:37:50,  7.86s/it]








collecting:  25%|██▌       | 254/1000 [36:20<1:38:40,  7.94s/it]








collecting:  26%|██▌       | 255/1000 [36:28<1:36:57,  7.81s/it]








collecting:  26%|██▌       | 256/1000 [36:36<1:37:20,  7.85s/it]








collecting:  26%|██▌       | 257/1000 [36:43<1:34:49,  7.66s/it]








collecting:  26%|██▌       | 258/1000 [36:51<1:35:12,  7.70s/it]








collecting:  26%|██▌       | 259/1000 [36:58<1:34:1

collecting:  33%|███▎      | 328/1000 [46:04<1:28:29,  7.90s/it]








collecting:  33%|███▎      | 329/1000 [46:12<1:27:58,  7.87s/it]








collecting:  33%|███▎      | 330/1000 [46:19<1:25:47,  7.68s/it]








collecting:  33%|███▎      | 331/1000 [46:26<1:23:22,  7.48s/it]








collecting:  33%|███▎      | 332/1000 [46:35<1:27:22,  7.85s/it]








collecting:  33%|███▎      | 333/1000 [46:42<1:26:12,  7.76s/it]








collecting:  33%|███▎      | 334/1000 [46:50<1:25:46,  7.73s/it]








collecting:  34%|███▎      | 335/1000 [46:58<1:25:21,  7.70s/it]








collecting:  34%|███▎      | 336/1000 [47:05<1:23:24,  7.54s/it]








collecting:  34%|███▎      | 337/1000 [47:14<1:28:46,  8.03s/it]








collecting:  34%|███▍      | 338/1000 [47:21<1:24:53,  7.69s/it]








collecting:  34%|███▍      | 339/1000 [47:28<1:24:51,  7.70s/it]








collecting:  34%|███▍      | 340/1000 [47:36<1:23:53,  7.63s/it]








collecting:  34%|███▍      | 341/1000 [47:43<1:22:4

...

collecting:  65%|██████▌   | 652/1000 [1:30:44<45:12,  7.79s/it]








collecting:  65%|██████▌   | 653/1000 [1:30:52<45:01,  7.79s/it]








collecting:  65%|██████▌   | 654/1000 [1:30:59<44:34,  7.73s/it]








collecting:  66%|██████▌   | 655/1000 [1:31:08<46:32,  8.09s/it]








collecting:  66%|██████▌   | 656/1000 [1:31:16<45:20,  7.91s/it]








collecting:  66%|██████▌   | 657/1000 [1:31:23<44:34,  7.80s/it]








collecting:  66%|██████▌   | 658/1000 [1:31:31<44:43,  7.85s/it]








collecting:  66%|██████▌   | 659/1000 [1:31:39<43:39,  7.68s/it]








collecting:  66%|██████▌   | 660/1000 [1:31:46<43:04,  7.60s/it]








collecting:  66%|██████▌   | 661/1000 [1:31:53<42:39,  7.55s/it]








collecting:  66%|██████▌   | 662/1000 [1:32:01<42:42,  7.58s/it]








collecting:  66%|██████▋   | 663/1000 [1:32:08<41:57,  7.47s/it]








collecting:  66%|██████▋   | 664/1000 [1:32:16<41:52,  7.48s/it]








collecting:  66%|██████▋   | 665/1000 [1:32:25<43:5

collecting:  73%|███████▎  | 734/1000 [1:41:16<33:17,  7.51s/it]








collecting:  74%|███████▎  | 735/1000 [1:41:26<35:42,  8.08s/it]








collecting:  74%|███████▎  | 736/1000 [1:41:34<35:21,  8.04s/it]








collecting:  74%|███████▎  | 737/1000 [1:41:42<35:06,  8.01s/it]








collecting:  74%|███████▍  | 738/1000 [1:41:49<34:35,  7.92s/it]








collecting:  74%|███████▍  | 739/1000 [1:41:57<34:05,  7.84s/it]








collecting:  74%|███████▍  | 740/1000 [1:42:05<34:17,  7.92s/it]








collecting:  74%|███████▍  | 741/1000 [1:42:12<33:27,  7.75s/it]








collecting:  74%|███████▍  | 742/1000 [1:42:19<32:29,  7.56s/it]








collecting:  74%|███████▍  | 743/1000 [1:42:27<32:28,  7.58s/it]








collecting:  74%|███████▍  | 744/1000 [1:42:35<32:27,  7.61s/it]








collecting:  74%|███████▍  | 745/1000 [1:42:44<33:47,  7.95s/it]








collecting:  75%|███████▍  | 746/1000 [1:42:51<33:17,  7.86s/it]








collecting:  75%|███████▍  | 747/1000 [1:42:59<33:4

collecting:  82%|████████▏ | 816/1000 [1:51:49<23:46,  7.75s/it]








collecting:  82%|████████▏ | 817/1000 [1:51:57<23:48,  7.80s/it]








collecting:  82%|████████▏ | 818/1000 [1:52:05<24:21,  8.03s/it]








collecting:  82%|████████▏ | 819/1000 [1:52:13<23:57,  7.94s/it]








collecting:  82%|████████▏ | 820/1000 [1:52:22<24:30,  8.17s/it]








collecting:  82%|████████▏ | 821/1000 [1:52:29<23:20,  7.83s/it]








collecting:  82%|████████▏ | 822/1000 [1:52:37<23:54,  8.06s/it]








collecting:  82%|████████▏ | 823/1000 [1:52:47<24:48,  8.41s/it]








collecting:  82%|████████▏ | 824/1000 [1:52:57<26:43,  9.11s/it]








collecting:  82%|████████▎ | 825/1000 [1:53:05<25:22,  8.70s/it]








collecting:  83%|████████▎ | 826/1000 [1:53:12<23:42,  8.17s/it]








collecting:  83%|████████▎ | 827/1000 [1:53:21<24:16,  8.42s/it]








collecting:  83%|████████▎ | 828/1000 [1:53:30<24:36,  8.58s/it]








collecting:  83%|████████▎ | 829/1000 [1:53:40<25:5

collecting:  90%|████████▉ | 898/1000 [2:02:33<12:49,  7.54s/it]








collecting:  90%|████████▉ | 899/1000 [2:02:41<12:48,  7.61s/it]








collecting:  90%|█████████ | 900/1000 [2:02:48<12:34,  7.54s/it]








collecting:  90%|█████████ | 901/1000 [2:02:56<12:31,  7.59s/it]








collecting:  90%|█████████ | 902/1000 [2:03:03<12:18,  7.54s/it]








collecting:  90%|█████████ | 903/1000 [2:03:11<12:15,  7.58s/it]








collecting:  90%|█████████ | 904/1000 [2:03:18<12:06,  7.57s/it]








collecting:  90%|█████████ | 905/1000 [2:03:26<12:10,  7.69s/it]








collecting:  91%|█████████ | 906/1000 [2:03:33<11:35,  7.40s/it]








collecting:  91%|█████████ | 907/1000 [2:03:40<11:30,  7.43s/it]








collecting:  91%|█████████ | 908/1000 [2:03:48<11:14,  7.34s/it]








collecting:  91%|█████████ | 909/1000 [2:03:56<11:49,  7.80s/it]








collecting:  91%|█████████ | 910/1000 [2:04:04<11:36,  7.74s/it]








collecting:  91%|█████████ | 911/1000 [2:04:11<11:0

collecting:  98%|█████████▊| 980/1000 [2:13:45<02:48,  8.41s/it]








collecting:  98%|█████████▊| 981/1000 [2:13:53<02:38,  8.33s/it]








collecting:  98%|█████████▊| 982/1000 [2:14:01<02:26,  8.16s/it]








collecting:  98%|█████████▊| 983/1000 [2:14:09<02:18,  8.13s/it]








collecting:  98%|█████████▊| 984/1000 [2:14:17<02:09,  8.11s/it]








collecting:  98%|█████████▊| 985/1000 [2:14:25<02:01,  8.08s/it]








collecting:  99%|█████████▊| 986/1000 [2:14:33<01:52,  8.02s/it]








collecting:  99%|█████████▊| 987/1000 [2:14:42<01:47,  8.27s/it]








collecting:  99%|█████████▉| 988/1000 [2:14:49<01:37,  8.09s/it]








collecting:  99%|█████████▉| 989/1000 [2:14:57<01:26,  7.89s/it]








collecting:  99%|█████████▉| 990/1000 [2:15:04<01:18,  7.88s/it]








collecting:  99%|█████████▉| 991/1000 [2:15:12<01:08,  7.65s/it]








collecting:  99%|█████████▉| 992/1000 [2:15:19<01:01,  7.63s/it]








collecting:  99%|█████████▉| 993/1000 [2:15:27<00:5

In [89]:
import pickle


with open('pu_keywords.pickle', 'wb') as handle:
    pickle.dump(pu_keywords, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [118]:
with open('pu_keywords.pickle', 'rb') as handle:
    pu = pickle.load(handle)

In [90]:
len(pu_keywords)

345

### 하룻동안 노출이 가장 많이된 top 1000 ref_term으로 봤을때, ref_term이 파워업 keyword에 포함된경우  
#### :345개

In [121]:
pu_keywords = pu

In [123]:
for k in pu_keywords:
    if len(pu_keywords[k]) > 0:
        pu_ids = ','.join('\''+str(i)+'\'' for i in pu_keywords[k]['pu_id'].unique())
        q = '''
        select pu_id, log_data
        from ad_power_up_log
        where log_type = 'add' and pu_id in ({})
        '''.format(pu_ids)
        temp = pd.read_sql(q, con=mysql)
        for pu_id in temp['pu_id'].tolist():
            pu_ids = []
            budgets = []
            pu_ids.append(pu_id)
            parse_json = eval(temp[temp['pu_id'] == pu_id]['log_data'].values[0])
            budgets.append(parse_json['pay_sum'])
        pu_budget = pd.DataFrame.from_dict({'pu_id': pu_ids, 'budget': budgets})
        pu_keywords[k]['pu_id'] = pu_keywords[k]['pu_id'].astype(str)
        pu_budget['pu_id'] = pu_budget['pu_id'].astype(str)
        pu_keywords[k] = pd.merge(pu_keywords[k], pu_budget, on='pu_id')

In [124]:
# before inserting budget
pu['스투시']

,pu_id,bid,cnt,imp_cnt,rank,budget
0,104607,80,32,33467,0,10000


In [125]:
# after inserting budget
pu_keywords['스투시']

,pu_id,bid,cnt,imp_cnt,rank,budget
0,104607,80,32,33467,0,10000


In [126]:
with open('pu_keywords_with_budget.pickle', 'wb') as handle:
    pickle.dump(pu_keywords, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [97]:
pu_keywords.keys()

dict_keys(['스투시', '롤', '자켓', '캠핑용품', '세탁기', '18k목걸이', '고프로', '지갑', '화장대', '톰브라운 가디건', '2060', '갤럭시폴드', '갤럭시노트20', '버즈 라이브', '블랙박스', '문상', '겐조', '이지부스트350', '스니커즈', '이지부스트', '버버리', '금목걸이', '투피스', '디올', '14k팔찌', '세븐틴', 'v50s', '발렌티노 클러치', '핸드폰', '아이패드', '꼼데가르송', 's9', '컴퓨터', '로렉스', '아이패드 프로 3세대', '루이비통 지갑', 'bts', '노트북', 's7', '명품가방', '삼성노트북', '식탁', '아이폰 8', '닌텐도 스위치', '몽클레어', '아노락', '빈티지', '루이비통 반지갑', '천안', '사카이', '향수', '갤럭시', '가디건', '킥보드', '구찌 라이톤', '원주', '구찌 반지갑', '책', '새상품', 'v40', '1060', '순금', '반팔', '아이폰 se', '볼캡', '갤럭시 워치', '스마트워치', 'rtx', '노트20', '아이폰11', '카드지갑', '스마트폰', '알렉산더맥퀸 오버솔', '갤럭시 버즈 플러스', '애플워치', 's20', '금팔찌', '남자 청바지', '갤럭시s10', '슬리퍼', '카트라이더', '안산', 's10e', '대전', '후드티', '구찌 스니커즈', '갤럭시탭', '광주', '방탄', '무료나눔', '버즈', '갤럭시s9', '아이패드 프로', '구찌 지갑', '라코스테 가디건', '전동킥보드', '게이밍노트북', '크롬하츠', '노트20울트라', '방탄소년단', '버즈플러스', '맥북', '구찌', '아이폰xs', '셔츠', '쇼파', '자동차', '나이키 스투시', '조거팬츠', '갤럭시버즈플러스', '오토바이', '루이비통 카드지갑', '루이비통', '컨버스', '톰브라운 클러치', '1660', '버즈 플러스', 'ssd', '알렉산더맥퀸', '아이폰11p